[![Abrir no Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/flavioluizseixas/llm-first-steps/blob/main/Ex3.ipynb)

## 🟠 3. Chatbot com Memória de Conversa (Histórico)

**🔹 Enunciado:**  
Crie um chatbot com histórico de interação, permitindo manter o contexto das conversas anteriores entre usuário e assistente.

**🧩 Implementação:**  
O prompt é construído dinamicamente com o histórico de pares (usuário, resposta). A interface foi construída com `gr.Blocks()` e inclui botão de limpar, `gr.Chatbot()` e controle de estado com `gr.State`.

**🚀 Resultados Esperados:**
- Mantém contexto conversacional entre perguntas.
- Interface simula uma conversa contínua com o assistente.

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
import gradio as gr

In [ ]:
# Carrega o modelo
model_id = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32,
    device_map="auto"
)

In [ ]:
# Função de resposta com histórico
def responder(mensagem, historico):
    # Constrói o prompt com todo o histórico
    prompt = "<|system|>Você é um assistente útil que responde sempre em português de forma clara e objetiva.<|end|>\n"
    for user_input, resposta in historico:
        prompt += f"<|user|>{user_input}<|end|>\n<|assistant|>{resposta}<|end|>\n"
    prompt += f"<|user|>{mensagem}<|end|>\n<|assistant|>"

    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

    outputs = model.generate(
        **inputs,
        max_new_tokens=150,
        temperature=0.7,
        top_p=0.9,
        do_sample=True,
        pad_token_id=tokenizer.eos_token_id,
        eos_token_id=tokenizer.eos_token_id
    )

    saida = tokenizer.decode(outputs[0], skip_special_tokens=True)
    resposta = saida.split("<|assistant|>")[-1].strip().split("<|end|>")[0].strip()

    historico.append((mensagem, resposta))
    return "", historico

In [ ]:
# Interface Gradio com Chatbot
with gr.Blocks() as demo:
    gr.Markdown("## 🤖 Assistente em Português com TinyLlama")
    chatbot = gr.Chatbot()
    entrada = gr.Textbox(label="Digite sua pergunta", placeholder="Ex: Quem descobriu o Brasil?")
    limpar = gr.Button("Limpar")
    estado = gr.State([])  # histórico

    entrada.submit(responder, [entrada, estado], [entrada, chatbot])
    limpar.click(lambda: ([], ""), None, [chatbot, estado])

demo.launch()
